In [1]:
import tensorflow as tf

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
import os
import pandas as pd
import datetime
import warnings
import network
import numpy as np
import func
import fnmatch
from params import dresden_csv, ins_train_csv, ins_test_csv, ins_train, ins_test, \
                ins_patches_db, ins_weights
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback

In [4]:
for path in [ins_train, ins_test]:
    if not os.path.exists(path):
        os.makedirs(path)

data = pd.read_csv(dresden_csv)
train_model = ['Ixus70', 'D200', 'mju-1050SW']
train_data = data[([m in train_model for m in data['model']])]

ins_data = train_data[(train_data['instance']==0)]
test_data = train_data[(train_data['instance']!=0)]

for db, csv in zip([ins_data, test_data], [ins_train_csv, ins_test_csv]):
    brand_model_list = []
    for brand, model in zip(db['brand'].values, db['model'].values):
        brand_model_list.append('_'.join((brand, model)))

    df = pd.DataFrame({'brand_model': brand_model_list, 'path': db['filename'].values})
    df.to_csv(csv, index=False)
    print('Saving db to csv')

Saving db to csv
Saving db to csv


In [5]:
images_db = pd.read_csv(ins_train_csv)

model_list = np.unique(images_db['brand_model'])
img_list = images_db['path']

if not os.path.exists(ins_patches_db):
    train_list, val_list, test_list, info, weights = func.split(img_list, model_list, ins_patches_db)
else:
    patches_db = np.load(ins_patches_db, allow_pickle=True).item()
    train_list = patches_db['train']
    val_list = patches_db['val']
    test_list = patches_db['test']
    info, weights = func.split_info(train_list, val_list, test_list, 
                                       model_list, total=len(img_list))

# store class weight to csv
df = pd.DataFrame([weights], columns=range(len(model_list)))
df.to_csv(ins_weights, index=False)

Canon_Ixus70 in training set: 112.
Canon_Ixus70 in validation set: 36.
Canon_Ixus70 in test set: 39.

Nikon_D200 in training set: 239.
Nikon_D200 in validation set: 59.
Nikon_D200 in test set: 74.

Olympus_mju-1050SW in training set: 138.
Olympus_mju-1050SW in validation set: 27.
Olympus_mju-1050SW in test set: 39.



In [6]:
test = pd.read_csv(ins_test_csv)
test_info = []
for m in model_list:
    tmp = fnmatch.filter(test['path'].values, m + '*')
    test_info.append(tmp)
    print('Instance test for camera {} has {} images.'.format(m, len(tmp)))

for i in range(len(model_list)):
    func.patch(model_list[i], info[i][0], 'train', patches_root=ins_train)
    func.patch(model_list[i], info[i][1], 'val' , patches_root=ins_train)
    func.patch(model_list[i], info[i][2], 'test', patches_root=ins_train)
    func.patch(model_list[i], test_info[i], '.', patches_root=ins_test)

100%|██████████| 112/112 [00:00<00:00, 170450.67it/s]

Instance test for camera Canon_Ixus70 has 380 images.
Instance test for camera Nikon_D200 has 380 images.
Instance test for camera Olympus_mju-1050SW has 836 images.



100%|██████████| 836/836 [00:00<00:00, 483846.85it/s]


In [7]:
img_height = 256
img_width = 256
batch_size = 64

train_generator = ImageDataGenerator(preprocessing_function=None,
    rescale=1./255, horizontal_flip=True, vertical_flip=True)

validation_generator = ImageDataGenerator(preprocessing_function=None,
    rescale=1./255)

train_data_gen = train_generator.flow_from_directory(
    directory=r"./instance/train/train/",
    target_size=(img_width, img_height), color_mode='grayscale',
    batch_size=batch_size, class_mode="categorical", shuffle=True)

validation_data_gen = validation_generator.flow_from_directory(
    directory=r"./instance/train/val/",
    target_size=(img_width, img_height), color_mode='grayscale',
    batch_size=batch_size, class_mode="categorical", shuffle=True)

Found 12227 images belonging to 3 classes.
Found 3050 images belonging to 3 classes.


In [ ]:
# Load the training and validation datasets
print("[*] Define model")
model = network.build()

sgd = tf.optimizers.SGD(lr=0.001, momentum=0.9, decay=0.0005)
model.compile(
    optimizer=sgd, 
    loss='categorical_crossentropy', 
    metrics=['accuracy'])

#           ------------ Train the Model ------------
if not os.path.exists('./instance/saved_model'):
    os.makedirs('./instance/saved_model')
    
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

ConstrainLayer = network.ConstrainLayer(model)
callbacks = [ModelCheckpoint('./instance/saved_model/weights.{epoch:02d}.h5',
    monitor='acc',verbose=1, save_best_only=False,
    save_freq=1), ConstrainLayer, tensorboard_callback]

df = pd.read_csv(ins_weights)
class_weight = df.to_dict('records')[0]
class_weight = {int(k):v for k, v in class_weight.items()}

history = model.fit_generator(generator=train_data_gen, epochs=45, workers=10,
     callbacks=callbacks, validation_data=validation_data_gen, class_weight=class_weight)

In [ ]:
model.save('./instance/model.h5')